In [1]:
#IMAGE_DIMS = (50,34,50,1)
IMAGE_DIMS = (224,152,224,1)
#IMAGE_DIMS = (112,76,112,1)
OUTPUT_DIR = '../../../output/kaggle-bowl/step10/'
#INPUT_DIR = '../../../input/step5-50/'
INPUT_DIR = '../../../input/step5-224/'
#INPUT_DIR = '../../../input/step5-112/'
DATASET_NAME_SUFFIX = '-centered-rotated'
LOAD_MODEL_FILE = None
#LOAD_MODEL_FILE = OUTPUT_DIR + 'tf-checkpoint-best5556'

In [2]:
import csv
import h5py
import numpy as np # linear algebra
import os
import logging
import tflearn

from modules.logging import logger
import modules.logging
import modules.lungprepare as lungprepare
import modules.utils as utils
import modules.cnn as cnn
from modules.utils import Timer

In [3]:
def start_training(model, input_dir, dataset_name_suffix, image_dims, output_dir, batch_size=50, n_epoch=10):

    utils.mkdirs(output_dir, recreate=False)

    modules.logging.setup_file_logger(output_dir + 'out.log')
    
    dataset_path = utils.dataset_path(input_dir, 'train' + dataset_name_suffix, IMAGE_DIMS)
    with h5py.File(dataset_path, 'r') as train_hdf5:
        X = train_hdf5['X']
        Y = train_hdf5['Y']
        logger.info('X shape ' + str(X.shape))
        logger.info('Y shape ' + str(Y.shape))

        dataset_path = utils.dataset_path(input_dir, 'validate' + dataset_name_suffix, image_dims)
        with h5py.File(dataset_path, 'r') as validate_hdf5:
            X_validate = validate_hdf5['X']
            Y_validate = validate_hdf5['Y']
            logger.info('X_validate shape ' + str(X_validate.shape))
            logger.info('Y_validate shape ' + str(Y_validate.shape))

            logger.info('Starting CNN training...')
            model.fit(X, Y, validation_set=(X_validate, Y_validate), 
                      shuffle=True, batch_size=batch_size, n_epoch=n_epoch,
                      show_metric=True, 
                      run_id='simplest1-'+str(image_dims))

In [4]:
logger.info('Prepare CNN for training')
network = cnn.net_simplest1(IMAGE_DIMS)
#network = cnn.net_deepmedic_simple(IMAGE_DIMS)
# network = cnn.net_alzheimer_cnn(IMAGE_DIMS)
model = cnn.prepare_cnn_model(network, OUTPUT_DIR, model_file=None)

2017-03-09 01:25:13,899 INFO Prepare CNN for training
2017-03-09 01:25:13,958 INFO Prepare CNN
2017-03-09 01:25:13,959 INFO Preparing output dir
2017-03-09 01:25:13,960 INFO Initializing network...


In [ ]:
logger.info('Train CNN')
start_training(model, INPUT_DIR, DATASET_NAME_SUFFIX, IMAGE_DIMS, OUTPUT_DIR, batch_size=8, n_epoch=5)
model.save(OUTPUT_DIR + 'final')
logger.info('==== ALL DONE ====')

Training Step: 1  | time: 11.508s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 008/538


In [10]:
logger.info('Evaluate model from dataset')
dataset_path = utils.dataset_path(INPUT_DIR, 'test' + DATASET_NAME_SUFFIX, IMAGE_DIMS)
cnn.evaluate_dataset(dataset_path, model)

2017-03-08 22:12:24,576 INFO Evaluate model from dataset
2017-03-08 22:12:24,580 DEBUG X_test shape (764, 112, 76, 112, 1)
2017-03-08 22:12:24,581 DEBUG Y_test shape (764, 2)
2017-03-08 22:12:24,582 INFO Evaluate performance on dataset ../../../input/step5-112/test-centered-rotated-112-76-112.h5...
2017-03-08 22:12:57,432 INFO Accuracy: [0.058900524574424587]
